In [104]:
# Import des packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

In [105]:
# URL de la page souhaitée

url = "https://www.sustainalytics.com/esg-ratings"

In [106]:
# Initialisation du Driver afin de pouvoir scraper sur Google Chrome
driver = webdriver.Chrome()

# Ouverture de l'URL
driver.get(url)

# Création des listes recevant les noms des entreprises et leurs scores ESG
grades = []
names = []

# Boucle permettant de scraper les X premières entreprises
for i in range(50):
    grades.append([grade.text for grade in driver.find_elements(By.CLASS_NAME, "col-2")])
    names.append([name.text for name in driver.find_elements(By.CSS_SELECTOR, "a.primary-color.d-block.js-fix-path")])

    print(f"{names[i]}: {grades[i]}")

    clickable = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, f"a.pagination-page[href='hrefcurrentpage={i+2}']"))
        )

    driver.execute_script("arguments[0].click();", clickable)

    time.sleep(3)

driver.quit()

KeyboardInterrupt: 

In [64]:
total_name = 0
total_grade = 0
for i in range(len(names)):
    total_name += len(names[i])
    total_grade += len(grades[i])
print(f"Au total, nous avons {total_name} entreprises et {total_grade} notes ESG")

Au total, nous avons 500 entreprises et 500 notes ESG


In [88]:
data = {}
for i in range(len(names)):
    for j in range(len(names[i])):
        if names[i][j] not in data:
            data[names[i][j]] = []
        data[names[i][j]].append(grades[i][j])

# Convertir ce dictionnaire en un DataFrame
df = pd.DataFrame.from_dict(data, orient='index')
df.rename(columns={df.columns[0]: 'Note ESG'}, inplace=True)
df.to_csv('data/esg_grades.csv', index=True, sep='\t')  # Le paramètre 'index=True' permet de sauvegarder les indices (noms des entreprises)

df

,Note ESG
"1-800-FLOWERS.COM, Inc.",22.6
1&1 AG,27.7
"10X Genomics, Inc.",22.5
11 Bit Studios SA,16.3
1st Source Corp.,35.0
...,...
"Allison Transmission Holdings, Inc.",29.4
Allo Issuer LLC,31.3
"Allogene Therapeutics, Inc.",25.2
Allos SA,23.0


In [107]:
driver = webdriver.Chrome()

# Ouverture de l'URL
driver.get(url)
def web_scraping(pages, sector):



    # Sélection du secteur
    select_element = Select(driver.find_element(By.ID, "industry")) 
    select_element.select_by_visible_text(sector)

    time.sleep(2)

    # Cliquer sur le bouton Filtrer afin d'appliquer le filtre
    filter_button = driver.find_element(By.ID, "filterButton")
    # Simuler le clique
    filter_button.click()

    time.sleep(2)

    # Création des listes recevant les noms et notes ESG des entreprises
    grades = []
    names = []

    # Boucle qui va scraper sur les différentes pages
    for i in range(pages):
        grades.append([grade.text for grade in driver.find_elements(By.CLASS_NAME, "col-2")])
        names.append([name.text for name in driver.find_elements(By.CSS_SELECTOR, "a.primary-color.d-block.js-fix-path")])

        # On met un "try" pour que le code fonctionne même si la page suivante n'existe pas
        try:
            
            next_page = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, f"a.pagination-page[href='hrefcurrentpage={i+2}']"))
            )
            driver.execute_script("arguments[0].click();", next_page)
            time.sleep(3)
        except:
            # Si le lien n'est pas trouvé (fin de pagination), on sort de la boucle
            print("Pas de page suivante, fin de la pagination.")
            break



    data = {}
    for x in range(len(names)):
        for y in range(len(names[x])):
            if names[x][y] not in data:
                data[names[x][y]] = []
            data[names[x][y]].append(grades[x][y])

    # Convertir ce dictionnaire en un DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    df.rename(columns={df.columns[0]: 'Note ESG'}, inplace=True)
    df.to_csv(f'data/{sector}.csv', index=True, sep='\t')  # Le paramètre 'index=True' permet de sauvegarder les indices (noms des entreprises)
    print(f'Le DataFrame {sector} a bien été enregistré')

sector_list = [
    "Aerospace & Defense",
    "Auto Components",
    "Automobiles",
    "Banks",
    "Building Products",
    "Chemicals",
    "Commercial Services",
    "Construction & Engineering",
    "Construction Materials",
    "Consumer Durables",
    "Consumer Services",
    "Containers & Packaging"
]

for i in range(len(sector_list)):
    web_scraping(2, sector_list[i])
driver.quit()

Le DataFrame Aerospace & Defense a bien été enregistré
Le DataFrame Auto Components a bien été enregistré
Le DataFrame Automobiles a bien été enregistré
Le DataFrame Banks a bien été enregistré
Le DataFrame Building Products a bien été enregistré
Le DataFrame Chemicals a bien été enregistré
Le DataFrame Commercial Services a bien été enregistré
Le DataFrame Construction & Engineering a bien été enregistré
Le DataFrame Construction Materials a bien été enregistré
Le DataFrame Consumer Durables a bien été enregistré
Le DataFrame Consumer Services a bien été enregistré
Le DataFrame Containers & Packaging a bien été enregistré
